In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="prod-organize-arizon-4e1c0a83")

query = """
SELECT
  c.pctnum
  , c.county
  , d.congressionaldistrict
  , d.statehousedistrict
  , sq.SurveyQuestionName
	, sr.SurveyResponseName
  , COUNT(cw.DWID) AS response_count

FROM `prod-organize-arizon-4e1c0a83.ngpvan.CTARAA_ContactsSurveyResponses_VF` AS cs

LEFT JOIN `prod-organize-arizon-4e1c0a83.ngpvan.CTARAA_SurveyQuestions` AS sq
	ON cs.SurveyQuestionID = sq.SurveyQuestionID

LEFT JOIN `prod-organize-arizon-4e1c0a83.ngpvan.CTARAA_SurveyResponses` AS sr
	ON cs.SurveyResponseID = sr.SurveyResponseID

LEFT JOIN `prod-organize-arizon-4e1c0a83.raze_raza_dwid_to_van_id_crosswalk.dwid_vanid_crosswalk` AS cw
  ON cs.VanID = cw.VanID

LEFT JOIN `proj-tmc-mem-mvp.catalist_cleaned.cln_catalist__district` AS d
  ON cw.DWID = d.dwid

INNER JOIN `prod-organize-arizon-4e1c0a83.rich_christina_proj.catalist_pctnum_crosswalk_native` AS c
  ON d.uniqueprecinctcode = c.uniqueprecinctcode

WHERE sq.Cycle ='2024'
	AND c.county in ('COCONINO','COCHISE','MOHAVE','PINAL','PIMA','YAVAPAI','YUMA')

GROUP BY 1,2,3,4,5,6
ORDER BY 1
"""

df = client.query(query).to_dataframe()

In [ ]:
df.head()

In [ ]:
# Prep data to send results back to project dataset
# Combine question and response into one column for pivoting
df['question_response'] = df['SurveyQuestionName'] + ' — ' + df['SurveyResponseName']

# Pivot
pivot_df = df.pivot_table(
    index=['pctnum', 'county', 'congressionaldistrict', 'statehousedistrict'],
    columns='question_response',
    values='response_count',
    fill_value=0
).astype(int).reset_index()

# Sanitize column names for BigQuery
pivot_df.columns = (
    pivot_df.columns
    .str.replace(r'[^\w]+', '_', regex=True)  # Replace anything not a-zA-Z0-9_ with _
    .str.strip('_')                          # Remove leading/trailing _
    .str.lower()                             # Optional: lowercase
)

pivot_df.head()

In [ ]:
from pandas_gbq import to_gbq

to_gbq(
    pivot_df,
    destination_table="rich_christina_proj.gotvid_counts_pivot",
    project_id="prod-organize-arizon-4e1c0a83",
    if_exists='replace'
)
